In [1]:
%matplotlib inline

import pandas as pd, numpy as np
import urllib
import json

import gdal
from gdalconst import * 
import os, sys, time

In [2]:
# function that extracts weather data from prism website

base_url = "http://services.nacse.org/prism/data/public/4km/"

months   = {1:'01', 2: '02', 3:'03', 4:'04', 5:'05', 6:'06', 7:'07', 8:'08', 9: '09',
               10:'10', 11:'11', 12:'12'}
days     = {1: 31, 2: 28, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9: 30,
               10:31, 11:30, 12:31}
lydays   = {1: 31, 2: 29, 3:31, 4:30, 5:31, 6:30, 7:31, 8:31, 9: 30,
               10:31, 11:30, 12:31}
def leapYear(year):
    if (year%400 == 0):
        return True
    elif (year%4 == 0) & (year%100 != 0):
        return True
    else:
        return False
    
#start year is 1994
def getDailyWeatherZip(var):
    for year in np.arange(1)+1999:
        for m in np.arange(2)+11:
            month = months[m]
            if leapYear(year):
                ndays = lydays[m]
            else:
                ndays = days[m]
            for d in np.arange(ndays)+1:
                if d < 10:
                    day = '0' + str(d)
                else:
                    day = str(d)
                date = str(year) + month + day
                #creating the url
                endpoint_url = base_url + var + '/' + date
                #opening a connection
                request = urllib.urlopen(endpoint_url)
                #save
                output = open("../ZipFiles2/"+ var +'/'+ date + ".zip", "w")
                output.write(request.read())
                output.close()
                time.sleep(3)

In [3]:
# getDailyWeatherZip('tmin')
# getDailyWeatherZip('tmax')
# getDailyWeatherZip('ppt')

In [4]:
import zipfile

def getDailyWeatherRaster(var):
    for year in np.arange(4)+1996:
        for m in np.arange(12)+1:
            month = months[m]
            if leapYear(year):
                ndays = lydays[m]
            else:
                ndays = days[m]
            for d in np.arange(ndays)+1:
                if d < 10:
                    day = '0' + str(d)
                else:
                    day = str(d)
                date = str(year) + month + day
                # unzipping a file
                fh = open("/Users/Alan/Desktop/ZipFiles/"+ var +'/'+ date + ".zip", "rb")
                z = zipfile.ZipFile(fh)
                # saving only the .bil file
                for name in z.namelist():
                    outpath = "/Users/Alan/Desktop/RasterFiles/" + var +'/'+ date
                    z.extract(name, outpath)
                fh.close()

In [5]:
# var = 'tmax'
# year = 1993
# month = '12'

# for d in np.arange(5)+27:
#     if d < 10:
#         day = '0' + str(d)
#     else:
#         day = str(d)
#     date = str(year) + month + day
#     # unzipping a file
#     fh = open("/Users/Alan/Desktop/ZipFiles/"+ var +'/'+ date + ".zip", "rb")
#     z = zipfile.ZipFile(fh)
#     # saving only the .bil file
#     for name in z.namelist():
#         outpath = "/Users/Alan/Desktop/RasterFiles/" + var +'/'+ date
#         z.extract(name, outpath)
#     fh.close()

In [6]:
# getDailyWeatherRaster('tmin')
# getDailyWeatherRaster('tmax')
# getDailyWeatherRaster('ppt')

In [7]:
def getValues(rasterDS, locData):
    size = len(locData)
    xValues = np.array(locData.loc[:, 'longitude'])
    yValues = np.array(locData.loc[:, 'latitude'])
    rows = rasterDS.RasterYSize #number of rows
    cols = rasterDS.RasterXSize # number of columns
    bands = rasterDS.RasterCount # number of data value types (it's going to be 1 for our case)
    
    # get georeference info
    transform = rasterDS.GetGeoTransform()
    xOrigin = transform[0]
    yOrigin = transform[3]
    pixelWidth = transform[1]
    pixelHeight = transform[5]
    
    band = rasterDS.GetRasterBand(1)
    # getting all data at once
    allData = band.ReadAsArray(0,0, cols, rows)
    
    #Result list
    result = []
    
    for i in range(size):
        # get x, y
        x = xValues[i]
        y = yValues[i]
        # compute pixel offset
        xOffset = int((x - xOrigin) / pixelWidth)
        yOffset = int((y - yOrigin) / pixelHeight)
        value   = allData[yOffset, xOffset]
        result.append(value)
    
    return result

In [8]:
base_file = '/Users/Alan/Desktop/RasterFiles/'
prism_base1 = '_stable_4kmD1_'
prism_base2 = '_stable_4kmD2_'
def getDailyWeatherData(var, cfile, prism_base):
    df = pd.DataFrame(index= [1], columns=np.arange(370)+1)
    df.columns = ['UID', 'latitude', 'longitude', 'Year'] + list(np.arange(366)+1)
    for year in np.arange(12)+1988:
        startTime = time.time() #if timing the function speed is really necessary
        data = cfile.copy()
        data['Year'] = year
        day_count = 1
        for m in np.arange(12)+1:
            month = months[m]
            if leapYear(year):
                ndays = lydays[m]
            else:
                ndays = days[m]
            for d in np.arange(ndays)+1:
                if d < 10:
                    day = '0' + str(d)
                else:
                    day = str(d)
                date = str(year) + month + day
                # raster file
                file_name = base_file + var +'/'+ date +'/'+ 'PRISM_' + var + prism_base + date + "_bil.bil"
                ds = gdal.Open(file_name)
                if ds is None:
                    print 'could not open ' + file_name
                    sys.exit(1)
                # weather data extracted using getValues function
                dailyData = getValues(ds, cfile)
                data[day_count] = dailyData
                day_count += 1
        if day_count < 365:
            print 'Error, check and see whether you have all the required files'
            sys.exit(1)
        elif day_count == 365:
            data[366] = None
        else:
            pass
        endTime = time.time()
        print 'The script took ' + str(endTime - startTime) + 'seconds'   
        df = pd.concat([df, data])
    return df

In [9]:
scoord = pd.read_csv('../modelDatasets/rm_seg_coords.csv')
cfile = scoord[['UID', 'latitude', 'longitude']]
cfile.head()

,UID,latitude,longitude
0,1,47.978215,-122.169366
1,27,47.978414,-122.133338
2,43,47.958874,-122.102727
3,66,47.947539,-122.070079
4,81,47.922928,-122.062374


In [10]:
gdal.AllRegister()
dTMIN = getDailyWeatherData('tmin', cfile, prism_base1)
dTMAX = getDailyWeatherData('tmax', cfile, prism_base1)
dPPT = getDailyWeatherData('ppt', cfile, prism_base2)

The script took 41.4584319592seconds
The script took 42.2261199951seconds
The script took 29.7477779388seconds
The script took 30.6892769337seconds
The script took 29.6406629086seconds
The script took 29.3050069809seconds
The script took 29.698638916seconds
The script took 29.834512949seconds
The script took 30.2672848701seconds
The script took 29.9936900139seconds
The script took 37.1267809868seconds
The script took 32.7108950615seconds
The script took 34.0074689388seconds
The script took 29.3823730946seconds
The script took 29.8960120678seconds
The script took 29.380975008seconds
The script took 29.7066030502seconds
The script took 28.965225935seconds
The script took 30.7079119682seconds
The script took 30.097826004seconds
The script took 29.7405688763seconds
The script took 31.2049629688seconds
The script took 33.7216441631seconds
The script took 41.1531441212seconds
The script took 31.9382491112seconds
The script took 41.5423800945seconds
The script took 42.2767131329seconds
The sc

In [11]:
dtmin = dTMIN.iloc[1:,:]
dtmax = dTMAX.iloc[1:,:]
dppt = dPPT.iloc[1:,:]

In [12]:
dtmax.head()

,1,2,3,4,5,6,7,8,9,10,...,361,362,363,364,365,366,UID,Year,latitude,longitude
0,6.69,4.42,4.29,4.09,4.09,6.14,8.40,9.36,4.02,8.06,...,5.87,2.23,5.35,5.73,10.41,10.61,1,1988,47.978215,-122.169366
1,6.40,4.28,4.10,4.05,4.03,6.06,8.06,8.92,4.00,7.80,...,5.50,2.08,5.08,5.36,10.14,10.13,27,1988,47.978414,-122.133338
2,6.30,4.28,4.04,4.18,4.15,6.13,7.83,8.58,4.12,7.64,...,5.26,1.98,4.92,5.03,9.92,9.72,43,1988,47.958874,-122.102727
3,6.30,4.28,4.04,4.18,4.15,6.13,7.83,8.58,4.12,7.64,...,5.26,1.98,4.92,5.03,9.92,9.72,66,1988,47.947539,-122.070079
4,6.31,4.56,4.25,4.67,4.61,6.52,7.75,8.35,4.56,7.70,...,5.15,2.27,4.99,4.90,9.98,9.39,81,1988,47.922928,-122.062374


In [13]:
print len(dtmin) == len(dtmax)
print len(dtmax) == len(dppt)

True
True


In [14]:
def check4missing(ds):
    count =0
    for i in np.arange(365):
        t = np.array(ds.iloc[:,i])
        n = len(t[abs(t) > 43])
        if  n > 0:
            count+=n
    return count

In [15]:
# k = check4missing(dtmax)
# print k

In [16]:
# k = [i & j for i, j in zip(k1, k2)]
# print k

In [17]:
wdata = dtmin[['UID', 'Year', 'latitude', 'longitude']]
wdata.loc[:,'A0'] = None
wdata.loc[:,'A1'] = None
wdata.loc[:,'A2'] = None
wdata.loc[:,'A3'] = None
wdata.loc[:,'B1'] = None
wdata.loc[:,'B2'] = None
wdata.loc[:,'B3'] = None
wdata.loc[:,'C'] = None

/Users/Alan/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:260: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/Alan/anaconda2/lib/python2.7/site-packages/pandas/core/indexing.py:420: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [18]:
print len(wdata) == len(dtmin)
wdata.index = np.arange(len(wdata))
len(wdata)

True


23520

In [19]:
for i in np.arange(len(wdata)):
    maxt = np.array(dtmax.iloc[i, 0:366])
    mint = np.array(dtmin.iloc[i, 0:366])
    ppt = np.array(dppt.iloc[i, 0:366])
    a = mint[~np.isnan(mint)]
    b = maxt[~np.isnan(maxt)]
    c = ppt[~np.isnan(ppt)]
    m1 = [a <= 0]
    m2 = [c > 0]
    mask = [p & q for p,q in zip(m1, m2)]
    wdata.loc[i, 'A0'] = len(a[mask])
    wdata.loc[i, 'A1'] = len(a[a<= 0])
    wdata.loc[i, 'A2'] = len(a[(a > 0) & (a<= 5)])
    wdata.loc[i, 'A3'] = len(a[(a > 5) & (a<= 15)])
    wdata.loc[i, 'B1'] = len(b[(b > 25) & (b<= 30)])
    wdata.loc[i, 'B2'] = len(b[(b > 30) & (b<= 35)])
    wdata.loc[i, 'B3'] = len(b[(b > 35)])
    wdata.loc[i, 'C'] = sum(c)
#     print i

In [20]:
wdata.head(10)

,UID,Year,latitude,longitude,A0,A1,A2,A3,B1,B2,B3,C
0,1,1988,47.978215,-122.169366,5,32,91,242,19,5,0,917.99
1,27,1988,47.978414,-122.133338,6,34,103,229,19,5,0,976.62
2,43,1988,47.958874,-122.102727,7,36,111,219,19,4,0,1062.94
3,66,1988,47.947539,-122.070079,7,36,111,219,19,4,0,1062.94
4,81,1988,47.922928,-122.062374,7,31,116,219,19,7,0,1131.9
5,104,1988,47.906457,-122.031866,7,31,116,219,19,7,0,1131.9
6,125,1988,47.882398,-122.007115,5,28,117,221,26,10,0,1200.42
7,142,1988,47.865028,-121.968844,5,30,118,217,25,10,0,1253.45
8,188,1988,47.854771,-121.884002,7,31,127,207,27,11,1,1367.81
9,226,1988,47.860322,-121.802218,7,31,129,205,30,10,2,1653.92


In [21]:
wdata.tail(10)

,UID,Year,latitude,longitude,A0,A1,A2,A3,B1,B2,B3,C
23510,72676,1999,47.468669,-117.609860,66,154,96,108,33,28,3,421.11
23511,72704,1999,47.499930,-117.564706,67,154,95,109,35,27,2,415.74
23512,72724,1999,47.499930,-117.564610,67,154,95,109,35,27,2,415.74
23513,72742,1999,47.528506,-117.579962,56,141,101,113,38,25,1,389.4
23514,72797,1999,47.183819,-120.887696,100,172,86,104,34,25,0,585.2
23515,72818,1999,47.178184,-120.847996,102,170,88,104,34,26,0,571.51
23516,72835,1999,47.183548,-120.812206,108,174,96,94,35,14,0,630.35
23517,72857,1999,47.194763,-120.771369,105,167,94,101,33,24,0,559.97
23518,72928,1999,47.800728,-120.203753,64,139,91,119,43,26,1,318.06
23519,72974,1999,47.854797,-120.197808,63,136,88,128,37,37,4,264.04


In [23]:
wdata.to_csv('../modelDatasets/weather88_99.csv')